<a href="https://colab.research.google.com/github/GallegoLab/PyF2F-Ruler/blob/main/PyF2F_Ruler_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to PyF2F-Ruler <br> 
In this tutorial, we provide the code to run:

1. **Image Registration** to correct for the chromatic shift in the two-channel images during the adquisition. 

2. **Distance Estimation** workflow to determine the distance between two fluorescent markers.

This notebook runs through an `example` dataset provided in our [GitHub](https://github.com/GallegoLab/PyF2F-Ruler). We provide instructions to run this code with another dataset below. <br><br>
For more details, checkout our [GitHub](https://github.com/GallegoLab/PyF2F-Ruler).

In [ ]:
#@title Install PyF2F-Ruler

#@markdown Please execute this cell by pressing the _Play_ button
#@markdown on the left to download and install PyF2F-Ruler software
#@markdown in this Colab notebook. 

#@markdown **Note**: This installs the software on the Colab
#@markdown notebook in the cloud and not on your computer.
%env PYTHONPATH = 
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_23.1.0-1-Linux-x86_64.sh
! chmod +x Miniconda3-py37_23.1.0-1-Linux-x86_64.sh
! bash ./Miniconda3-py37_23.1.0-1-Linux-x86_64.sh -b -f -p /usr/local/
!conda install --channel defaults conda python=3.7 --yes


# Clone github and install requirements to run PyF2F-Ruler
!git clone https://github.com/GallegoLab/PyF2F-Ruler
%cd PyF2F-Ruler
!pip install -r requirements.txt
!conda install ipykernel --yes
!pip install protobuf==3.20.*  # to make tensorflow work properly
!pip install zenodo-get        # to get zenodo weights.zip file



## Running PyF2F-Ruler

Please set the `parameters` required to run the **image registration**
and **distance estimation** workflows. Then hit `Runtime` -> `Run after`.<br>

You can also run the cells individually by pressing the *Play* button on the left. <br>

You can find a detailed explanation about PyF2F-Ruler's workflow and a step-by-step tutorial in the [GitHub](https://github.com/GallegoLab/PyF2F-Ruler). 

In [ ]:
#@title Set parameters to run PyF2F ⬇️
#@markdown Please set the `parameters` required to run the **image registration**
#@markdown and **distance estimation** workflows:
#@markdown ###**Parameters:**
# , then hit `Runtime` -> `Run all`
import os
if not os.getcwd() == "/content/PyF2F-Ruler/scripts/":
  os.chdir("/content/PyF2F-Ruler/scripts/")
job_name = 'example'                    #@param {type:"string"}
pixel_size = 64.5                  #@param [64.5, 110] {type:"raw"}
# Define spot diameter
if pixel_size == 64.5:
  spot_diameter = int(11)
elif pixel_size == 110:
  spot_diameter = int(7)

# spot detection: sort spots below this percentile of intensity 
percentile = 99.8                  #@param {type:"raw"}
# linking: link spots from ch1-ch2 channels separated by this cutoff in px
max_displacement = 2               #@param {type:"raw"}

#@markdown **Image Registration:** 
beads_head = '*.tif'               #@param {type:"string"}
beads_reg_dir = 'reg'               #@param {type:"string"}
beads_test_dir = 'test'             #@param {type:"string"}
# Create directories for output
out_reg_dir = 'out_reg'         #@param {type:"string"}
out_test_dir = 'out_test'       #@param {type:"string"}
# Bead detection based on light properties
# spot detection: sort spots with a mass above this threshold (range 0-1). 
min_mass = 0.01                    #@param {type:"raw"}
# spot detection: sort spots with a mass below this threshold (range 0-1).
max_mass =0.95                     #@param {type:"raw"}

#@markdown **Distance Estimation :**
rolling_ball_radius = 70                    #@param {type:"raw"}
median_filter = 10                          #@param {type:"raw"}
contour_cutoff = 13                         #@param {type:"raw"}
neigh_cutoff = 9                            #@param {type:"raw"}
kde_cutoff = 0.5                            #@param {type:"raw"}
gaussian_cutoff = 0.35                      #@param {type:"raw"}
mle_cutoff = 0.66                           #@param {type:"raw"}

#@markdown Select the following checkbox to perform a the image registration with a 
#@markdown global or local affine transformation.

# Select between 'global' or 'local' registration 
global_registration = True #@param {type:"boolean"}
local_registration = False #@param {type:"boolean"}
# local registration: max distance in px for nearest-neighbour search
search_range = 2000 // pixel_size  #local registration: max distance in px for nearest-neighbour search
min_fiducials = 10                 #local registration: minimum number of fiducial markers to correct locally 

#@markdown 
#@markdown 
#@markdown You may want to track the spot detection and selection along the 
#@markdown process. The following checkbox will generate HTML outputs for each 
#@markdown step of the process (will run slower).
do_html_plots = True #@param {type:"boolean"}

# Paths to directories where Beads_reg and Beads_test
path_beads_reg = f"../{job_name}/input/{beads_reg_dir}/"
path_beads_test = f"../{job_name}/input/{beads_test_dir}/"

path_output_reg = f"../{job_name}/input/{out_reg_dir}/"
path_output_test = f"../{job_name}/input/{out_test_dir}/"

if not os.path.exists(path_output_reg):
    os.mkdir(path_output_reg)
    
if not os.path.exists(path_output_test):
    os.mkdir(path_output_test)

# Get weights.zip file from Zenodo repo with record ID 3598690
!zenodo_get -r 3598690
!unzip weights.zip
!rm weights.zip



In [ ]:
#@title 1. Run Image Registration protocol 
if global_registration:
  if do_html_plots:
    !python3 PyF2F_image_registration.py -bri $path_beads_reg -bti $path_beads_test -bro $path_output_reg -bto $path_output_test --name $beads_head -px $pixel_size -sd $spot_diameter -pc $percentile --min_mass $min_mass --max_mass $max_mass -l $max_displacement --global --plot
  else:
    !python3 PyF2F_image_registration.py -bri $path_beads_reg -bti $path_beads_test -bro $path_output_reg -bto $path_output_test --name $beads_head -px $pixel_size -sd $spot_diameter -pc $percentile --min_mass $min_mass --max_mass $max_mass -l $max_displacement --global
if local_registration:
  if do_html_plots:
    !python3 PyF2F_image_registration.py -bri $path_beads_reg -bti $path_beads_test -bro $path_output_reg -bto $path_output_test --name $beads_head -px $pixel_size -sd $spot_diameter -pc $percentile --min_mass $min_mass --max_mass $max_mass -l $max_displacement -s $search_range --min_fiducials $min_fiducials --plot
  else:
    !python3 PyF2F_image_registration.py -bri $path_beads_reg -bti $path_beads_test -bro $path_output_reg -bto $path_output_test --name $beads_head -px $pixel_size -sd $spot_diameter -pc $percentile --min_mass $min_mass --max_mass $max_mass -l $max_displacement -s $search_range --min_fiducials $min_fiducials  

In [ ]:
#@title 2. Download Image Registration Results 
from google.colab import files
!zip -r /content/PyF2F-Ruler/example/input/registration.zip /content/PyF2F-Ruler/example/input/$out_test_dir
files.download('/content/PyF2F-Ruler/example/input/registration.zip') 

In [ ]:
#@title 3. Run Distance Estimation protocol
if global_registration:
  if do_html_plots:
    !python3 Pyf2f_main.py -d ../$job_name/ --px_size $pixel_size --rolling_ball_radius $rolling_ball_radius --median_filter $median_filter --particle_diameter $spot_diameter --percentile $percentile --max_displacement $max_displacement --contour_cutoff $contour_cutoff --neigh_cutoff $neigh_cutoff --kde_cutoff $kde_cutoff --gaussian_cutoff $gaussian_cutoff --mle_cutoff $mle_cutoff --dirty
  else:
    !python3 Pyf2f_main.py -d ../$job_name/ --px_size $pixel_size --rolling_ball_radius $rolling_ball_radius --median_filter $median_filter --particle_diameter $spot_diameter --percentile $percentile --max_displacement $max_displacement --contour_cutoff $contour_cutoff --neigh_cutoff $neigh_cutoff --kde_cutoff $kde_cutoff --gaussian_cutoff $gaussian_cutoff --mle_cutoff $mle_cutoff 
if local_registration:
  if do_html_plots:
    !python3 Pyf2f_main.py -d ../$job_name/ --px_size $pixel_size --rolling_ball_radius $rolling_ball_radius --median_filter $median_filter --particle_diameter $spot_diameter --percentile $percentile --max_displacement $max_displacement --contour_cutoff $contour_cutoff --neigh_cutoff $neigh_cutoff --kde_cutoff $kde_cutoff --gaussian_cutoff $gaussian_cutoff --mle_cutoff $mle_cutoff --local_transformation --dirty
  else:
    !python3 Pyf2f_main.py -d ../$job_name/ --px_size $pixel_size --rolling_ball_radius $rolling_ball_radius --median_filter $median_filter --particle_diameter $spot_diameter --percentile $percentile --max_displacement $max_displacement --contour_cutoff $contour_cutoff --neigh_cutoff $neigh_cutoff --kde_cutoff $kde_cutoff --gaussian_cutoff $gaussian_cutoff --mle_cutoff $mle_cutoff --local_transformation

In [ ]:
#@title 4. Package and Download All Results 
from google.colab import files
!zip -r /content/PyF2F-Ruler/results.zip /content/PyF2F-Ruler/example/
files.download(f'/content/PyF2F-Ruler/results.zip') 

# Instructions
**Quick start**
1. Set parameters for the image registration and distance estimation workflows.
2. Press `Runtime` -> `Run after`.
3. The pipeline consists of 4 steps. The currently running step is indicated by a circle with a stop sign next to it.

**Running the *example* dataset**

The pipeline will run through the '*example*' dataset used in [Picco et al., 2017](https://www.sciencedirect.com/science/article/pii/S0092867417300521) which corresponds to the distance between the Sec5-GFP (exocyst subunit) and the anchor Sla2-RFP-FKBP, using Exo70-FRB as bait. The dataset is cloned from our [GitHub repository](https://github.com/GallegoLab/PyF2F-Ruler).<br><br>

The '*example*' folder has the following structure:<br>

*example*/
> *input*/
>>    *pict_images*/ --> where the PICT images are.  <br>
>>    *reg*/ --> Beads set to create the registration map.  <br>
>>    *test*/ --> Beads set to test the image registration.  <br>


**Result zip file contents**

At the end of the job, the *example/output/* directory is packed and downloaded as *results.zip*. In this folder you will find the following subfolders:

*   *images/* contains the pre-processed images.
*   *spots/*: contains the data from spot detection on your PICT images.
*   *segmentation/*: contains the segmented images, masks, and contour images.
*   *results/*: contains the resulting files from each processing/analysis step
*   *figures/*: contains HTML and png files to get track of the detected and selected spots for each image, on each step, as well as the distance distribution for each step. It also contains PDF file with the final distance distribution and the $μ$ and $σ$ estimations.<br>